<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [19]:
from pathlib import Path
import pandas as pd
import spacy
import gensim
from gensim import corpora
from gensim.models import LdaMulticore

In [2]:
nlp = spacy.load("en_core_web_lg")
path = Path("data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

In [3]:
df = pd.read_csv(path)

In [4]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [5]:
reviews = df['reviews.text']
reviews.head()

0    I order 3 of them and one of the item is bad q...
1    Bulk is always the less expensive way to go fo...
2    Well they are not Duracell but for the price i...
3    Seem to work as well as name brand batteries a...
4    These batteries are very long lasting the pric...
Name: reviews.text, dtype: object

In [6]:
STOP_WORDS = nlp.Defaults.stop_words.union([
    'batteries','I', 'amazon', 'i', 'Amazon', 'it', "it's", 'it.', 'the', 'this', 'battery'])

In [7]:
nlp.remove_pipe("parser")
nlp.remove_pipe("tagger")
nlp.Defaults.stop_words.update(STOP_WORDS)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True


In [8]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc 
           if token.is_stop != True 
           and token.is_punct != True
           and token.text != "/n"
          ]
    return nlp.make_doc(doc)

def remove_persons(doc):
    # This takes in a doc of tokens from the NER and removes any person entities
    
    persons = set(
        [ent.text for ent in doc.ents
         if str(ent.label_) == "PERSON"])

    doc = [token.text for token in doc if token.text not in persons]

    doc = u' '.join(doc)
    return nlp.make_doc(doc)

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',before='ner')
# nlp.add_pipe(remove_persons,name='person_remover',after='ner')
# nlp.add_pipe(remove_stopwords, name="stopwords", before="ner")

In [9]:
def get_reviews(reviews: pd.Series):
    for review in reviews.iteritems():
        yield review[1]

In [10]:
docs_gen = nlp.pipe(get_reviews(reviews), n_process=4)

In [11]:
docs = [
        [token.text for token in doc 
           if token.is_stop != True 
           and token.is_punct != True
           and token.text != "/n"
          ]
     for doc in docs_gen]

In [12]:
docs[0]

['order',
 '3',
 'item',
 'bad',
 'quality',
 'miss',
 'backup',
 'spring',
 'pcs',
 'aluminum',
 'work']

### LDA Topic Modeling

In [13]:
id2word = corpora.Dictionary(docs)

In [14]:
id2word.filter_extremes(no_below=5, no_above=0.95)
len(id2word.keys())

3351

In [17]:
corpus = [id2word.doc2bow(text) for text in docs]

In [20]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 9,
                   passes=10,
                   workers=4
                  )

In [21]:
lda.print_topics()

[(0,
  '0.038*"tablet" + 0.029*"Kindle" + 0.028*"Fire" + 0.026*"app" + 0.020*"use" + 0.016*"purchase" + 0.014*"kindle" + 0.014*"read" + 0.013*"book" + 0.013*"screen"'),
 (1,
  '0.123*"great" + 0.103*"good" + 0.079*"price" + 0.048*"work" + 0.035*"Good" + 0.026*"tablet" + 0.025*"buy" + 0.023*"product" + 0.018*"Works" + 0.018*"quality"'),
 (2,
  '0.077*"tablet" + 0.063*"kid" + 0.027*"child" + 0.020*"great" + 0.020*"time" + 0.019*"love" + 0.018*"good" + 0.015*"case" + 0.014*"app" + 0.013*"control"'),
 (3,
  '0.041*"tablet" + 0.037*"like" + 0.020*"use" + 0.017*"game" + 0.016*"screen" + 0.016*"good" + 0.016*"play" + 0.015*"watch" + 0.013*"price" + 0.012*"little"'),
 (4,
  '0.205*"Great" + 0.092*"price" + 0.054*"product" + 0.028*"value" + 0.022*"expect" + 0.021*"Excellent" + 0.017*"good" + 0.016*"recommend" + 0.016*"item" + 0.016*"quality"'),
 (5,
  '0.071*"old" + 0.069*"year" + 0.064*"love" + 0.030*"game" + 0.025*"buy" + 0.025*"play" + 0.023*"2" + 0.023*"tablet" + 0.021*"use" + 0.019*"daught

In [23]:
import re
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
tablet Kindle Fire app use

------ Topic 1 ------
great good price work Good

------ Topic 2 ------
tablet kid child great time

------ Topic 3 ------
tablet like use game screen

------ Topic 4 ------
Great price product value expect

------ Topic 5 ------
old year love game buy

------ Topic 6 ------
charge device light Kindle use

------ Topic 7 ------
long brand use good work

------ Topic 8 ------
use love easy read book



## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling